#####Init

In [ ]:
!pip install wandb
!wandb login
# c33c1aa1f13cf760137464172f5cec9bd819b1c7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 26.1 MB/s 
     |████████████████████████████████| 158 kB 67.4 MB/s 
     |████████████████████████████████| 181 kB 68.1 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 157 kB 70.9 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 76.6 MB/s 
     |████████████████████████████████| 157 kB 54.9 MB/s 
     |████████████████████████████████| 157 kB 71.8 MB/s 
     |████████████████████████████████| 157 kB 54.2 MB/s 
     |████████████████████████████████| 157 kB 72.9 MB/s 
     |████████████████████████████████| 156 kB 62.7 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=b56a3f07b96be73aa1cc1e0c506cc1713d2f2a1ec262b0fcffa9d6390ce93ac5
  Stored in directory: /root/.cache/pip/wheels/3e/31

In [ ]:
%cd drive/MyDrive
%ls

/content/drive/MyDrive
 농산물1.ipynb       'Colab Notebooks'/   submit_AAA.csv   wandb/
 answer_example.csv   model_output/       submit.csv       wnb/
 aT_data/             NongLinear.ipynb    submit_I.csv


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
import torch.backends.cudnn as cudnn

import os
import numpy as np
import pandas as pd
import math
from tqdm.auto import tqdm
from glob import glob
import random
import wandb
import time
import warnings
import torch.backends.cudnn as cudnn
from sklearn.model_selection import train_test_split

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(seed)
warnings.filterwarnings(action='ignore')

#####Datasets

In [ ]:
from pandas.core.resample import Day
def time_window(df, t_interval):
    n_samples = len(df) - t_interval + 1
    result = []
    for index in range(n_samples):
        tmp = df[index: index + t_interval]
        tmp = np.vstack(tmp).astype(np.float32)
        result.append(tmp)
    return np.array(result)

def time_feature(df):
    month = []
    day_of_week = []
    week = []
    day = []
    quarter = []

    for i in range(len(df)):
        date = df[i]

        day.append(date.day)
        month.append(date.month)
        day_of_week.append(date.day_of_week)
        week.append(date.week)
        quarter.append(date.quarter)
    
    day_np = np.array(day).reshape(-1, 1)
    month_np = np.array(month).reshape(-1, 1)
    day_of_week_np = np.array(day_of_week).reshape(-1, 1)
    week_np = np.array(week).reshape(-1, 1)
    quarter_np = np.array(quarter).reshape(-1, 1)

    features_np = np.concatenate([day_np, month_np, day_of_week_np, week_np, quarter_np], axis=1)
    return features_np

In [ ]:
class NongTrainDataset(Dataset):
    # 해당 데이터에서는 날짜와 가격만을 데이터로 가진다.
    def __init__(self, data_path='./aT_data/data/train', item_idx=0, use_val=False):
        super(NongTrainDataset, self).__init__()
        # 파일 경로
        file_path = os.path.join(data_path, f'train_{item_idx}.csv')

        # csv 파일 읽기
        data_df = pd.read_csv(file_path)

        ## 데이터 전처리
        # 빈 값은 np.nan으로 변경
        for column in data_df.columns:
            data_df[column] = data_df[column].replace({' ': np.nan})
        
        # np.nan은 0으로 변경
        data_df = data_df.fillna(0)

        # 사용할 데이터 선정
        used_columns_list = ['해당일자_전체평균가격(원)', 'datadate']
        inputs_df = data_df[[column for column in data_df.columns if column in used_columns_list]]
        labels_df = data_df[used_columns_list[0]]

        # labels_df에서 처음 14개의 값은 사용하지 않는다.
        labels_df = labels_df[14:]

        # 시간 정보 추가
        inputs_df['datadate'] = pd.to_datetime(inputs_df['datadate'].astype('str'))
        time_stamp_np = time_feature(inputs_df['datadate'])

        inputs_df.drop('datadate', axis=1, inplace=True)
        inputs_np = np.concatenate([inputs_df.values, time_stamp_np], axis=1)

        labels_np = labels_df.values

        # np.array로 변환
        inputs_np = time_window(inputs_np, 14)
        labels_np = time_window(labels_np, 28)

        inputs_np = inputs_np[:len(labels_np)]

        # torch.tensor로 변환
        self.inputs = torch.tensor(inputs_np, dtype=torch.float32)
        self.labels = torch.tensor(labels_np, dtype=torch.float32)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]
    
    def __len__(self):
        return len(self.inputs)

d = NongTrainDataset()

class NongTestDataset(Dataset):
    def __init__(self, data_path='./aT_data/data/test', item_idx=0):
        super(NongTestDataset, self).__init__()

        zero_csv = [0 for i in range(14)]

        inputs_np_list = []

        for set_idx in range(10):
            # 파일 경호
            file_path = os.path.join(data_path, f'set_{set_idx}/test_{item_idx}.csv')

            # csv 파일 읽기
            data_df = pd.read_csv(file_path)

            if len(data_df) == 0:
                data_df['zero_non'] = zero_csv
                data_df = data_df.fillna(0)
                data_df.drop('zero_non', axis=1, inplace=True)
                data_df.drop('Unnamed: 0', axis=1, inplace=True)

            # 빈 값은 np.nan으로 변환
            for column in data_df.columns:
                data_df[column] = data_df[column].replace({' ': np.nan})
            
            # np.nan은 0으로 변환
            data_df = data_df.fillna(0)

            # 사용할 feature 선택
            used_columns_list = ['해당일자_전체평균가격(원)', 'datadate']
            inputs_df = data_df[[column for column in data_df.columns if column in used_columns_list]]
            
            # 시간 정보 추가
            inputs_df['datadate'] = pd.to_datetime(inputs_df['datadate'].astype('str'))
            time_stamp_np = time_feature(inputs_df['datadate'])

            inputs_df.drop('datadate', axis=1, inplace=True)
            inputs_np = np.concatenate([inputs_df.values, time_stamp_np], axis=1)

            inputs_np_list.append(inputs_np)
        
        inputs_np_list = np.array(inputs_np_list, dtype=np.float32)
        self.inputs = torch.tensor(inputs_np_list, dtype=torch.float32)

    def __getitem__(self, idx):
        return self.inputs[idx]
    
    def __len__(self):
        return len(self.inputs)

#####Models

In [ ]:
import torch.nn as nn

# NLinear
class NLinear(nn.Module):
    def __init__(self, flags):
        super(NLinear, self).__init__()
        self.seq_len = flags['seq_len']
        self.pred_len = flags['pred_len']
        self.linear1 = nn.Linear(self.seq_len, 128, bias=True)
        self.linear2 = nn.Linear(128, 128, bias=True)
        self.linear3 = nn.Linear(128, self.pred_len, bias=True)
        self.mish = nn.Mish()

    def forward(self, x):
        seq_last = x[:, -1:, :].detach()

        x = x - seq_last
        x = self.linear1(x.permute(0, 2, 1))

        x = self.mish(x)
        x = self.linear2(x)
        x = self.mish(x)
        x = self.linear3(x).permute(0, 2, 1)
        x = self.mish(x)
        x = x + seq_last

        return x

flags = {
    'seq_len': 14,
    'pred_len': 28,
}

model = NLinear(flags)

In [ ]:
# DLinear
class moving_avg(nn.Module):
    """
    Moving average block to highlight the trend of time series
    """
    def __init__(self, kernel_size, stride):
        super(moving_avg, self).__init__()
        self.kernel_size = kernel_size
        self.avg = nn.AvgPool1d(kernel_size=kernel_size, stride=stride, padding=0)

    def forward(self, x):
        # padding on the both ends of time series
        front = x[:, 0:1, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        end = x[:, -1:, :].repeat(1, (self.kernel_size - 1) // 2, 1)
        x = torch.cat([front, x, end], dim=1)
        x = self.avg(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        return x

class series_decomp(nn.Module):
    """
    Series decomposition block
    """
    def __init__(self, kernel_size):
        super(series_decomp, self).__init__()
        self.moving_avg = moving_avg(kernel_size, stride=1)

    def forward(self, x):
        moving_mean = self.moving_avg(x)
        res = x - moving_mean
        return res, moving_mean

class DecompositionModule(nn.Module):
    def __init__(self, in_features, out_features):
        super(DecompositionModule, self).__init__()
        self.seq_len = in_features
        self.pred_len = out_features
        kernel_size = 25
        self.decomposition = series_decomp(kernel_size)
        self.linear_seasonal = nn.Linear(self.seq_len, self.pred_len)
        self.linear_trend = nn.Linear(self.seq_len, self.pred_len)
    
    def forward(self, x):
        seasonal_in, trend_in = self.decomposition(x)
        seasonal_out = self.linear_seasonal(seasonal_in.permute(0, 2, 1))
        trend_out = self.linear_trend(trend_in.permute(0, 2, 1))
        total_out = seasonal_out + trend_out
        return total_out.permute(0, 2, 1)

class MultiChannel_DLinear(nn.Module):
    def __init__(self, flags):
        super(MultiChannel_DLinear, self).__init__()
        self.seq_len = flags.seq_len
        self.pred_len = flags.pred_len
        self.n_channels = flags.n_channels

        self.dlinear = nn.ModuleList()

        for i in range(self.n_channels):
            self.dlinear.append(DecompositionModule(self.seq_len, self.pred_len))

        self.linear = nn.Linear(self.n_channels, 1)
    
    def forward(self, x):
        outputs = None

        for i in range(self.n_channels):
            output = self.dlinear[i](x[:, :, i:i+1])
            if outputs == None:
                outputs = output
            else:
                outputs = torch.cat([outputs, output], dim=2)
        
        return self.linear(outputs)


class DLinear(nn.Module):
    def __init__(self, flags):
        super(DLinear, self).__init__()
        self.seq_len = flags.seq_len
        self.pred_len = flags.pred_len
        self.n_layers = flags.n_layers

        self.layers = nn.ModuleList()
        for i in range(self.n_layers - 1):
            self.layers.append(DecompositionModule(self.seq_len, self.seq_len))
        self.layers.append(DecompositionModule(self.seq_len, self.pred_len))

        self.activation = nn.Mish()

    def forward(self, x):
        for i in range(self.n_layers):
            x = self.activation(self.layers[i](x))
        return x

In [ ]:
flags = {
    'seq_len': 14,
    'pred_len': 28,
    'n_channels': 4,
}

flags = Flag(flags)

m = MultiChannel_DLinear(flags)

inputs = torch.ones(64, 14, 4)

print(f'inputs: {inputs.shape}')
outputs = m(inputs)
print(f'outputs: {outputs.shape}')

inputs: torch.Size([64, 14, 4])
outputs: torch.Size([64, 28, 1])


In [ ]:
class DNLinear(nn.Module):
    def __init__(self, flags):
        super(DNLinear, self).__init__()
        
        kernel_size = 25
        self.decomposition = series_decomp(kernel_size)

        self.nlinear_seasonal = NLinear(flags)
        self.nlinear_trend = NLinear(flags)

    def forward(self, x):
        seasonal_in, trend_in = self.decomposition(x)
        seasonal_out = self.nlinear_seasonal(seasonal_in)
        trend_out = self.nlinear_trend(trend_in)
        return seasonal_out + trend_out

In [ ]:
class MultiChannel_DNLinear(nn.Module):
    def __init__(self, flags):
        super(MultiChannel_DNLinear, self).__init__()

        self.n_channels = flags.n_channels

        self.perchannel_dnlinear = nn.ModuleList()

        for i in range(self.n_channels):
            self.perchannel_dnlinear.append(DNLinear(flags))
        
        self.linear = nn.Linear(self.n_channels, 1)

    def forward(self, x):
        perchannel_outputs_list = []
        
        for i in range(self.n_channels):
            perchannel_outputs_list.append(self.perchannel_dnlinear[i](x[:, :, i:i+1]))
        
        total_outputs = torch.concat(perchannel_outputs_list, dim=2)

        return self.linear(total_outputs)


In [ ]:
def get_model(flags):
    if flags.model == 'nlinear':
        return NLinear(flags)
    if flags.model == 'dlinear':
        return DLinear(flags)
    if flags.model == 'dnlinear':
        return DNLinear(flags)
    if flags.model == 'multichannel_dnlinear':
        return MultiChannel_DNLinear(flags)
    if flags.model == 'multichannel_dlinear':
        return MultiChannel_DLinear(flags)

#####Utils

In [ ]:
class Flag:
    def __init__(self, flags):
        for key, value in flags.items():
            if isinstance(value, dict):
                self.__dict__[key] = Flag(value)
            else:
                self.__dict__[key] = value

In [ ]:
class Scheduler:
    def __init__(self, optimizer, flags, lr_lambda=None):
        if flags.train.lr_scheduler.method == 'lambda':
            if lr_lambda == None:
                print('lr_lambda must not be None.')
            self.scheduler = lr_scheduler.LambdaLR(
                optimizer=optimizer,
                lr_lambda=lr_lambda
            )
        elif flags.train.lr_scheduler.method == 'step':
            self.scheduler = lr_scheduler.StepLR(
                optimizer=optimizer,
                step_size=flags.train.lr_scheduler.step_size,
                gamma=flags.train.lr_scheduler.gamma
            )
        elif flags.train.lr_scheduler.method == 'linear':
            self.scheduler = lr_scheduler.LinearLR(
                optimizer=optimizer,
                start_factor=flags.train.lr_scheduler.start_factor,
                end_factor=flags.train.lr_scheduler.end_factor,
                total_iters=flags.train.lr_scheduler.total_iters
            )
        elif flags.train.lr_scheduler.method == 'exponential':
            self.scheduler = lr_scheduler.ExponentialLR(
                optimizer=optimizer,
                gamma=flags.train.lr_scheduler.gamma,
            )
        
    def step(self):
        self.scheduler.step()

#####Train

In [ ]:
def train(flags):
    # Exp Name
    name = f"{flags['exp_name']}_{flags['item_idx']}"

    # Wandb
    wandb.init(
        config=flags,
        project="Nong",
        entity="gradient-flow",
        name=name
    )

    # Flags
    flags = Flag(flags)

    # Dataset
    train_dataset = NongTrainDataset(item_idx=flags.item_idx)

    # Dataloader
    train_dataloader = DataLoader(
        dataset=train_dataset,
        batch_size=flags.batch_size,
        shuffle=True,
    )

    # Model
    model = get_model(flags.multichannel_dlinear)

    # Weight Initialization
    # for name, param in model.named_parameters():
    #     nn.init.xavier_uniform_(param)

    # Criterion
    criterion = nn.L1Loss()

    # Optimizer
    optimizer = optim.Adam(
        params=model.parameters(),
        lr=flags.train.lr,
        betas=(0.9, 0.999),
    )

    # Scheduler
    lr_lambda = lambda epoch: (flags.train.lr_scheduler.gamma) ** epoch
    scheduler = Scheduler(
        optimizer=optimizer,
        flags=flags,
        lr_lambda=lr_lambda
    )

    # Load Model to Device
    model = model.to(flags.device)
    # wandb.watch(model)

    # Train
    best_train_loss = 987654321.0
    
    print(f'Train for item -> {flags.item_idx}')
    for epoch in tqdm(range(flags.epochs)):
        model.train()

        train_loss = 0.0
        n_train_samples = len(train_dataloader.dataset)

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(flags.device), labels.to(flags.device)
            # inputs [batch, 14, 6]
            # outputs [batch, 28]
            
            # Predict
            outputs = model(inputs)

            # Loss
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * len(inputs)
        
        scheduler.step()

        train_loss = train_loss / n_train_samples
        wandb.log({'train_loss': train_loss})

        if best_train_loss > train_loss:
            best_train_loss = train_loss
            torch.save(model.state_dict(), f'./wnb/{flags.exp_name}/{flags.exp_name}_{flags.item_idx}.pth')
    print(f'Best Train Loss: {best_train_loss}')

In [ ]:
def train_all_items(flags):
    flags['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(f'Using {flags["device"]} Device')

    if flags['exp_name'] == None:
        exp_name = input('Exp name: ')
        flags['exp_name'] = exp_name
        
    wnb_path = f"./wnb/{flags['exp_name']}"

    if not os.path.isdir(wnb_path):
        os.mkdir(wnb_path)
    
    for item_idx in range(37):
        flags['item_idx'] = item_idx
        train(flags)

#####Experiment

In [ ]:
flags = {
    # General
    'exp_name': None,
    'item_idx': None,
    'batch_size': 64,
    'epochs': 100,
    # NLinear
    'nlinear': {
        'model': 'nlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 1,
    },
    # DLinear
    'dlinear': {
        'model': 'dlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 2,        
    },
    # MultiChannel_DLinear
    'multichannel_dlinear': {
        'model': 'multichannel_dlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_channels': 6,
    },
    # DNLinear
    'dnlinear': {
        'model': 'dnlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 2,
    },
    # MultiChannel_DNLinear
    'multichannel_dnlinear': {
        'model': 'multichannel_dnlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 1,
        'n_channels': 4,
    },
    # train
    'train': {
        'lr': 1e-3,
        'lr_scheduler': {
            'method': 'lambda',
            'gamma': 0.97
        }
    },
    'device': None,
}

if __name__ == '__main__':
    train_all_items(flags)

torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([12, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([12, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 

train_loss,█▇▄▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,833.04106


Train for item -> 26


  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([12, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 

...

torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([12, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 

train_loss,█▇▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss,497.51789


Train for item -> 36


  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([12, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 14, 6])
torch.Size([64, 


#####Test

In [ ]:
def test(flags):
    # Flags
    flags = Flag(flags)

    flags.exp_name = 'AAA'

    # Dataset
    test_dataset = NongTestDataset()

    # DataLoader
    test_dataloader = DataLoader(
        dataset=test_dataset,
        batch_size=1,
        shuffle=False
    )

    # Model
    model = get_model(flags.multichannel_dlinear)

    # Weight Load
    model.load_state_dict(torch.load(f'./wnb/{flags.exp_name}/{flags.exp_name}_{flags.item_idx}.pth'))

    # Load Model to Device
    model = model.to(flags.device)

    # Eval
    model.eval()

    for set_idx, inputs in enumerate(test_dataloader):
        inputs = inputs.to(flags.device)

        with torch.no_grad():
            outputs = model(inputs)
        
        outputs_np = outputs.cpu().detach().numpy().reshape(1, 28)
        outputs_df = pd.DataFrame(outputs_np).T
        outputs_df.to_csv(f'./model_output/set_{set_idx}/predict_{flags.item_idx}.csv', index=False)

In [ ]:
flags = {
    # General
    'exp_name': None,
    'item_idx': None,
    'batch_size': 64,
    'epochs': 100,
    # NLinear
    'nlinear': {
        'model': 'nlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 1,
    },
    # DLinear
    'dlinear': {
        'model': 'dlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 2,        
    },
    # MultiChannel_DLinear
    'multichannel_dlinear': {
        'model': 'multichannel_dlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_channels': 6,
    },
    # DNLinear
    'dnlinear': {
        'model': 'dnlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 2,
    },
    # MultiChannel_DNLinear
    'multichannel_dnlinear': {
        'model': 'multichannel_dnlinear',
        'seq_len': 14,
        'pred_len': 28,
        'n_layers': 1,
        'n_channels': 4,
    },
    # train
    'train': {
        'lr': 1e-3,
        'lr_scheduler': {
            'method': 'lambda',
            'gamma': 0.97
        }
    },
    'device': None,
}

def test_all_items(flags):
    for item_idx in tqdm(range(37)):
        if flags['device'] == None:
            flags['device'] = 'cuda' if torch.cuda.is_available() else 'cpu'
            
        if flags['exp_name'] == None:
            exp_name = input('Exp name: ')
            flags['exp_name'] = exp_name
        
        flags['item_idx'] = item_idx
        test(flags)

if __name__ == '__main__':
    test_all_items(flags)

  0%|          | 0/37 [00:00<?, ?it/s]

Exp name: AAB


#####Submit

In [ ]:
for k in range(10):
    globals()[f'set_df_{k}'] = pd.DataFrame()
    answer_df_list = glob(f'./model_output/set_{k}/*.csv') # 예측한 결과 불러오기
    pum_list = glob(f'./aT_data/aT_test_raw/sep_{k}/*.csv') # 기존 test input 불러오기
    pummok = [a for a in pum_list if 'pummok' in a.split('/')[-1]]

    for i in answer_df_list:
        df = pd.read_csv(i)
        number = i.split('_')[-1].split('.')[0]

        base_number = 0
        for p in pummok:
            if number == p.split('_')[-1].split('.')[0]:
                pum_df = pd.read_csv(p)

                if len(pum_df) != 0:
                    base_number = pum_df.iloc[len(pum_df)-1]['해당일자_전체평균가격(원)']  # 기존 각 sep 마다 test input의 마지막 target 값 가져오기 (변동률 계산을 위해)
                else:
                    base_number = np.nan
        globals()[f'set_df_{k}'][f'품목{number}']  = [base_number] + list(df[df.columns[-1]].values) # 각 품목당 순서를 t, t+1 ... t+28 로 변경
    globals()[f'set_df_{k}'] = globals()[f'set_df_{k}'][[f'품목{col}' for col in range(37)]] # 열 순서를 품목0 ~ 품목36 으로 변경

date = [f'd+{i}' for i in range(1,15)] + ['d+22 ~ 28 평균']

for k in range(10):
    globals()[f'answer_df_{k}'] = pd.DataFrame()
    for c in globals()[f'set_df_{k}'].columns:
        base_d = globals()[f'set_df_{k}'][c][0]

        ans_1_14 = []
        for i in range(14):
            ans_1_14.append((globals()[f'set_df_{k}'][c].iloc[i+1]- base_d)/base_d)  # t+1 ~ t+14 까지는 (t+n - t)/t 로 계산

        ans_22_28 = (globals()[f'set_df_{k}'][c][22:29].mean() - base_d)/base_d # t+22 ~ t+28은 np.mean(t+22 ~ t+28) - t / t

        globals()[f'answer_df_{k}'][f'{c} 변동률'] = ans_1_14 + [ans_22_28]
  
    globals()[f'answer_df_{k}']['Set'] = k # set 번호 설정
    globals()[f'answer_df_{k}']['일자'] = date # 일자 설정

# 위에서 계산된 변동률 들을 합쳐주는 과정

all_df =pd.DataFrame()
for i in range(10):
    if i== 0 :
        all_df = pd.concat([all_df, globals()[f'answer_df_{i}']],axis=1)
    else:
        all_df = pd.concat([all_df, globals()[f'answer_df_{i}']])

all_df = all_df[['Set','일자'] + list(all_df.columns[:-2])]
all_df.reset_index(drop=True, inplace=True)

# set, 일자 기억하기위해 따로 저장

re_set = list(all_df['Set'])
re_date = list(all_df['일자'])


# 정답 양식 불러오기
out_ans = pd.read_csv('./answer_example.csv')

# 두 dataframe 합치기 (nan + 숫자 = nan 이용)
submit_df = all_df + out_ans

submit_df['Set'] = re_set
submit_df['일자'] = re_date


# 최종 저장
submit_df.to_csv(f'./submit_{flags["exp_name"]}.csv',index=False)